In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, f1_score, roc_curve, precision_score, recall_score, accuracy_score, auc
import numpy as np
import datetime as dt
import os
from torch import sigmoid, Tensor

In [ ]:
%matplotlib inline

In [ ]:
pred_df = pd.read_csv('../data/all_preds_best_mean_inun_v2.csv')

In [ ]:
#pred_df['pred'] = sigmoid(Tensor(pred_df['pred'])) # If providing outputs before sigmoid activation

In [ ]:
# Add months and years back in
pred_df['date'] = np.tile(pd.date_range('1984-03', periods=418, freq='M'), int(pred_df.shape[0]/418))
pred_df = pred_df.set_index(['id','date'])

In [ ]:
pred_df['set_flag'] = 2
pred_df.loc[pred_df.index.get_level_values(1)<pd.Timestamp('2015-01-01'), 'set_flag'] = 1
pred_df.loc[pred_df.index.get_level_values(1)<pd.Timestamp('2011-01-01'), 'set_flag'] = 0

In [ ]:
test_pred_df = pred_df.loc[pred_df['set_flag']==2]
val_pred_df = pred_df.loc[pred_df['set_flag']==1]
train_pred_df = pred_df.loc[pred_df['set_flag']==0]

# Platt Scaling 
## Doesn't work very well

In [ ]:
# without overfit correction
from sklearn.linear_model import LogisticRegression as LR
# From: http://fastml.com/classifier-calibration-with-platts-scaling-and-isotonic-regression/
# lr = LR()                                                       
# lr.fit(train_pred_df['pred'].values.reshape(-1,1), train_pred_df['true'].values.reshape(-1,1))


# with overfit correction
# Would have to use a workaround and not sure it would make much difference
# train_pred_df['platt_target'] = 0
# tot_pos = train_pred_df['true'].sum()
# tot_neg = (train_pred_df['true']==0).sum()
# train_pred_df.loc[train_pred_df['true']==1] = (tot_pos+1)/(tot_pos+2)
# train_pred_df.loc[train_pred_df['true']==0] = (1)/(tot_neg+2)
# lr = LR()                                                       
# lr.fit(train_pred_df['pred'].values.reshape(-1,1), train_pred_df['platt_target'].values.reshape(-1,1))


In [ ]:
# pred_df['pred_cal'] = lr.predict_proba(pred_df['pred'].values.reshape( -1, 1 ))[:,1]
# train_pred_df['pred_cal'] = lr.predict_proba(train_pred_df['pred'].values.reshape( -1, 1 ))[:,1]
# val_pred_df['pred_cal'] = lr.predict_proba(val_pred_df['pred'].values.reshape( -1, 1 ))[:,1]
# test_pred_df['pred_cal'] = lr.predict_proba(test_pred_df['pred'].values.reshape( -1, 1 ))[:,1]

# Isotonic regression

In [ ]:
from sklearn.isotonic import IsotonicRegression as IR
# From: http://fastml.com/classifier-calibration-with-plats-scaling-and-isotonic-regression/
ir = IR( out_of_bounds = 'clip' )                                                       
ir.fit(val_pred_df['pred'].values, val_pred_df['true'].values)


In [ ]:
pred_df['pred_cal'] = ir.transform(pred_df['pred'].values)
train_pred_df['pred_cal'] = ir.transform(train_pred_df['pred'].values)
val_pred_df['pred_cal'] = ir.transform(val_pred_df['pred'].values)
test_pred_df['pred_cal'] = ir.transform(test_pred_df['pred'].values)
pred_df.loc[pred_df['pred_cal']<=0,'pred_cal'] = 0.00000000001
train_pred_df.loc[train_pred_df['pred_cal']<=0,'pred_cal'] = 0.00000000001
val_pred_df.loc[val_pred_df['pred_cal']<=0,'pred_cal'] = 0.00000000001
test_pred_df.loc[test_pred_df['pred_cal']<=0,'pred_cal'] = 0.00000000001

# Reliability plots

In [ ]:
nbins = 10
bin_size = 1/nbins
target_probs = np.arange(0.05, 1.00001, bin_size)

def calc_probs(cur_pred_df, bin_size, pred_col='pred'):
    actual_probs = []
    pred_probs = []
    for bin_start in np.arange(0, 1, bin_size):
        current_preds = cur_pred_df.loc[(cur_pred_df[pred_col] > bin_start) & (cur_pred_df[pred_col] < (bin_start + bin_size))]
        actual_probs.append(current_preds['true'].sum()/current_preds.shape[0])
        pred_probs.append(current_preds[pred_col].mean())
    return actual_probs, pred_probs

### All

In [ ]:
fig, ax = plt.subplots(1,2)
actual_probs, pred_probs = calc_probs(pred_df, bin_size)
actual_probs_cal, pred_probs_cal = calc_probs(pred_df, bin_size, 'pred_cal')
ax[0].scatter(pred_probs, actual_probs)
ax[0].plot([0, 1], [0, 1], 'k-')
ax[0].set_title('Before calibration')
ax[0].set_ylabel('P(Inundation)')
ax[0].set_xlabel('Confidence')
ax[1].scatter(pred_probs_cal, actual_probs_cal)
ax[1].plot([0, 1], [0, 1], 'k-')
ax[1].set_title('After calibration')
ax[1].set_ylabel('P(Inundation)')
ax[1].set_xlabel('Confidence')
fig.show()


### Train

In [ ]:
fig, ax = plt.subplots(1,2)
actual_train_probs, pred_train_probs = calc_probs(train_pred_df, bin_size)
actual_train_probs_cal, pred_train_probs_cal = calc_probs(train_pred_df, bin_size, 'pred_cal')
ax[0].scatter(pred_train_probs, actual_train_probs)
ax[0].plot([0, 1], [0, 1], 'k-')
ax[0].set_title('Before calibration')
ax[0].set_ylabel('P(Inundation)')
ax[0].set_xlabel('Confidence')
ax[1].scatter(pred_train_probs_cal, actual_train_probs_cal)
ax[1].plot([0, 1], [0, 1], 'k-')
ax[1].set_title('After calibration')
ax[1].set_ylabel('P(Inundation)')
ax[1].set_xlabel('Confidence')
fig.show()


### Validation

In [ ]:
fig, ax = plt.subplots(1,2)
actual_val_probs, pred_val_probs = calc_probs(val_pred_df, bin_size)
actual_val_probs_cal, pred_val_probs_cal = calc_probs(val_pred_df, bin_size, 'pred_cal')
ax[0].scatter(pred_val_probs, actual_val_probs)
ax[0].plot([0, 1], [0, 1], 'k-')
ax[0].set_title('Before calibration')
ax[0].set_ylabel('P(Inundation)')
ax[0].set_xlabel('Confidence')
ax[1].scatter(pred_val_probs_cal, actual_val_probs_cal)
ax[1].plot([0, 1], [0, 1], 'k-')
ax[1].set_title('After calibration')
ax[1].set_ylabel('P(Inundation)')
ax[1].set_xlabel('Confidence')
fig.show()

### Test

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(13,5))
actual_test_probs, pred_test_probs = calc_probs(test_pred_df, bin_size)
actual_test_probs_cal, pred_test_probs_cal = calc_probs(test_pred_df, bin_size, 'pred_cal')
ax[0].scatter(pred_test_probs, actual_test_probs)
ax[0].plot([0, 1], [0, 1], 'k-')
ax[0].set_title('Before calibration', size=14)
ax[0].set_ylabel('Actual P(Inundation)', size=13)
ax[0].set_xlabel('Model Confidence', size=13)
ax[1].scatter(pred_test_probs_cal, actual_test_probs_cal)
ax[1].plot([0, 1], [0, 1], 'k-')
ax[1].set_title('After calibration', size=14)
ax[1].set_ylabel('Actual P(Inundation)', size=13)
ax[1].set_xlabel('Model Confidence', size=13)
plt.savefig('/home/ksolvik/research/misc_projects/playa/deliverables/figures/test_calibration.png', dpi=300, bbox_inches='tight')
fig.show()

## Paper Figure

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(12,10))
# Train
ax[0,0].scatter(pred_train_probs, actual_train_probs)
ax[0,0].plot([0, 1], [0, 1], 'k-')
ax[0,0].set_title('Train, Before calibration', size=14)
ax[0,0].set_ylabel('Actual P(Inundation)', size=13)
ax[0,0].set_xlabel('Model Confidence', size=13)
ax[0,1].scatter(pred_train_probs_cal, actual_train_probs_cal)
ax[0,1].plot([0, 1], [0, 1], 'k-')
ax[0,1].set_title('Train, After calibration', size=14)
ax[0,1].set_ylabel('Actual P(Inundation)', size=13)
ax[0,1].set_xlabel('Model Confidence', size=13)
# Val
ax[1,0].scatter(pred_val_probs, actual_val_probs)
ax[1,0].plot([0, 1], [0, 1], 'k-')
ax[1,0].set_title('Validation, Before calibration', size=14)
ax[1,0].set_ylabel('Actual P(Inundation)', size=13)
ax[1,0].set_xlabel('Model Confidence', size=13)
ax[1,1].scatter(pred_val_probs_cal, actual_val_probs_cal)
ax[1,1].plot([0, 1], [0, 1], 'k-')
ax[1,1].set_title('Validation, After calibration', size=14)
ax[1,1].set_ylabel('Actual P(Inundation)', size=13)
ax[1,1].set_xlabel('Model Confidence', size=13)
# Test
ax[2,0].scatter(pred_test_probs, actual_test_probs)
ax[2,0].plot([0, 1], [0, 1], 'k-')
ax[2,0].set_title('Test, Before calibration', size=14)
ax[2,0].set_ylabel('Actual P(Inundation)', size=13)
ax[2,0].set_xlabel('Model Confidence', size=13)
ax[2,1].scatter(pred_test_probs_cal, actual_test_probs_cal)
ax[2,1].plot([0, 1], [0, 1], 'k-')
ax[2,1].set_title('Test, After calibration', size=14)
ax[2,1].set_ylabel('Actual P(Inundation)', size=13)
ax[2,1].set_xlabel('Model Confidence', size=13)
fig.tight_layout()
plt.savefig('/home/ksolvik/research/misc_projects/playa/deliverables/figures/test_calibration.png', dpi=300, bbox_inches='tight')
fig.show()

# Checking performance

In [ ]:
print(log_loss(train_pred_df['true'].values, train_pred_df['pred'].values))
print(log_loss(train_pred_df['true'].values, train_pred_df['pred_cal'].values.astype('float64')))

In [ ]:
print(log_loss( val_pred_df['true'].values, val_pred_df['pred'].values))
print(log_loss( val_pred_df['true'].values, val_pred_df['pred_cal'].values.astype('float64')))

In [ ]:
print(log_loss(test_pred_df['true'].values, test_pred_df['pred'].values))
print(log_loss(test_pred_df['true'].values, test_pred_df['pred_cal'].values.astype('float64')))

In [ ]:
cutoff=0.25
# print('train f1 (before, after):',
#       f1_score(train_pred_df['true'].values, train_pred_df['pred'].values>cutoff),
#       f1_score(train_pred_df['true'].values, train_pred_df['pred_cal'].values>cutoff))
print('val f1 (before, after):',
      f1_score(val_pred_df['true'].values, val_pred_df['pred'].values>cutoff),
      f1_score(val_pred_df['true'].values, val_pred_df['pred_cal'].values>cutoff))
print('test f1 (before, after):',
      f1_score(test_pred_df['true'].values, test_pred_df['pred'].values>cutoff),
      f1_score(test_pred_df['true'].values, test_pred_df['pred_cal'].values>cutoff))

 # Save results

In [ ]:
pred_df.to_csv('../data/all_preds_best_mean_inun_v2_calibrated.csv')